# 1. Convert from csv to txt

In [35]:
import pandas as pd

In [36]:
csv_file = "../data/hotel_with_nearest_restaurants.csv"
df = pd.read_csv(csv_file)
total_text = ""
for index, row in df.iterrows():
    string = ""
    hotel_name = row['hotel_name']
    total_room = row['total_rooms']
    admin = row['admin']
    hotel_price = row['hotel_price']
    city = row['city']
    address = row['address']
    map_link = row['map_link']
    hotel_feature = row['hotel_features']
    nearest_restaurant = row['nearest_restaurants']
    short_description = row['short_description']
    long_description = row['long_description']
    # In ra thông tin
    string += f"Khách sạn {hotel_name}, với tổng cộng {total_room} phòng, được quản lý bởi {admin}, có giá {hotel_price} đồng, nằm tại {city}, địa chỉ {address}, với đường dẫn bản đồ {map_link}, có các tiện ích {hotel_feature}, và có các nhà hàng gần nhất:\n{nearest_restaurant}, mô tả ngắn gọn: {short_description}, mô tả chi tiết: {long_description} \n"
    total_text += string
    # print(string)

In [37]:
txt_file = 'output.txt'
with open(txt_file, 'w', encoding='utf-8') as f:
    f.write(total_text)

# 2. Build a vector database based on FAISS

In [38]:
from dotenv import load_dotenv
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [39]:
load_dotenv('.env')

True

In [40]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("The OpenAI API key is not set. Please set the OPENAI_API_KEY environment variable.")
else:
    print("API key loaded successfully.")

API key loaded successfully.


In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=api_key)
model.invoke("Hello, world!")

In [ ]:
# Load tài liệu
loader = TextLoader("output.txt",encoding="utf-8")
documents = loader.load()

# Cắt nhỏ văn bản
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Tạo vector database (FAISS)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
vectorstore.save_local("faiss_index")

In [ ]:
embeddings = OpenAIEmbeddings()
loaded_vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

In [ ]:
results = loaded_vectorstore.similarity_search("Nhà hàng DeLaSea Hạ Long")
results

# 3. Pass retrieval data into the prompt

In [34]:
prompt_template = """
Bạn là một chuyên gia du lịch và có kiến thức đầy đủ về các khách sạn tại Việt Nam. Bạn hãy trả lời câu hỏi của tôi dựa trên thông tin mà tôi đã cung cấp cho bạn. Các câu hỏi chủ yếu về các địa điểm du lịch, và cách xây dựng tour du lịch hợp lí bao gồm khách sạn, nhà hàng cũng như các địa điểm du lịch nổi tiếng gần đó.
Dưới đây là thông tin về một khách sạn mà tôi đã tìm thấy: {results}

Câu hỏi của tôi là: {question}

Hãy cung cấp thông tin chi tiết về khách sạn và nhà hàng mà tôi yêu cầu, sử dụng các thông tin đã có và bổ sung thêm nếu có. Nếu có các thông tin về đường dẫn tới map hay facebook, website của khách sạn hay nhà hàng, hãy cung cấp cho tôi. Nếu không có thông tin nào, hãy nói là không có thông tin nào.
"""

In [ ]:
messages = model.invoke(prompt_template.format(results=results, question="Xây dựng cho tôi một tour du lịch 3 ngày 2 đêm tại Hạ Long, bao gồm khách sạn, nhà hàng và các địa điểm du lịch nổi tiếng gần đó."))
print(messages.content)

In [ ]:
messages.content